In [1]:

# Install a Drive FUSE wrapper.# Insta 
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [3]:

# Generate creds for the Drive FUSE library.# Gener 
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
# Work around misordering of STREAM and STDIN in Jupyter.
# https://github.com/jupyter/notebook/issues/3159
prompt = !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass(prompt[0] + '\n\nEnter verification code: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force

Enter verification code: ··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive

print ('Files in Drive:')
!ls drive/


Files in Drive:
1115-resume-powerpoint-template-(cv-template).pdf
4_6030726325214380780.odt
AP-Workshop-Session2.odt
Back transliteration.ods
cat.jpg
Colab Notebooks
created.txt
cross language survey.odt
ferdosi.txt
fra.txt
google_analogy2.txt
google_analogy.txt
Homeworks-G2.ods
IMG_20141110_092043~2.jpg
Marks.ods
myself.jpg
NER & Word Clustering.ods
New Doc 2.pdf
Open source Tookits or Frameworks or ....ods
outputs
Paper revision.odt
para2vec.odt
Persian corpus information.ods
portrait.jpg
portrait.png
presentation
resized_cat.jpg
resized_myself.jpg
resized_portrait.jpg
resized_starrynight.jpg
simlex999-translated.xlsx
simlex999-translated.xlsx.ods
simlex-final.ods
SNLP_PRESENTATION.pdf
SNLP_PRESENTATION.pptx
SourceCodes
starrynight.jpg
survey.odt
Theoretical Interview.pdf
transliteration.odt
university position.ods
Untitled document.odt
Untitled document.odt (75ca89bf)
Untitled presentation.pdf
Untitled presentation.pdf (58a532f6)
Useful things to know.ods
Word embedding.ods
آموزش ور

In [0]:
# -*- coding: utf-8 -*-
#
# Copyright 2018 Amir Hadifar. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
import numpy as np
import tensorflow.keras as keras

text = open('drive/ferdosi.txt').read()

unique = sorted(set(text))
vocab_size = len(unique)
# creating a mapping from unique characters to indices
char2idx = {u: i for i, u in enumerate(unique)}
idx2char = {i: u for i, u in enumerate(unique)}

input_text = []
target_text = []
max_length = 50
batch_size = 32

for f in range(0, len(text) - max_length, max_length):
    inps = text[f:f + max_length]
    targ = text[f + 1:f + 1 + max_length]

    input_text.append([char2idx[i] for i in inps])
    target_text.append([char2idx[t] for t in targ])

input_text = np.array(input_text)
target_text = np.array(target_text)


class MyModel(keras.Sequential):
    def __init__(self, vocab=vocab_size, batch_siz3=128, embed_size=128, rnn_dim=128, seq_len=40):
        super(MyModel, self).__init__()

        self.add(keras.layers.Embedding(vocab, embed_size))

        self.add(keras.layers.GRU(rnn_dim, return_sequences=True))

        self.add(keras.layers.Dense(vocab, activation='softmax'))


model = MyModel(batch_siz3=batch_size)
model.compile(optimizer=keras.optimizers.Nadam(0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(model.summary())
print(50 * '-')
print(model.get_config())

model.fit(x=input_text, y=np.expand_dims(target_text, 2), epochs=60, batch_size=batch_size)

num_generate = 1000

# You can change the start string to experiment
start_string = 'ب'
# converting our start string to numbers(vectorizing!)
# input_eval = [ for s in start_string]
# input_eval = np.expand_dims(input_eval, 0)
input_eval = np.zeros([max_length])
input_eval[-1] = char2idx[start_string]
# empty string to store our results
text_generated = ''

# low temperatures results in more predictable text.
# higher temperatures results in more surprising text
# experiment to find the best setting
temperature = 1.0
model.reset_states()

ids = [i for i in range(vocab_size)]
for i in range(num_generate):
    predictions = model.predict(input_eval)

    predicted_id = np.random.multinomial(1, predictions[0][0], 1).argmax()

    input_eval = np.roll(input_eval, 1)
    input_eval[-1] = predicted_id

    text_generated += idx2char[predicted_id]

print(start_string + text_generated)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         6016      
_________________________________________________________________
gru (GRU)                    (None, None, 128)         98688     
_________________________________________________________________
dense (Dense)                (None, None, 47)          6063      
Total params: 110,767
Trainable params: 110,767
Non-trainable params: 0
_________________________________________________________________
None
--------------------------------------------------
[{'class_name': 'Embedding', 'config': {'name': 'embedding', 'trainable': True, 'batch_input_shape': (None, None), 'dtype': 'float32', 'input_dim': 47, 'output_dim': 128, 'embeddings_initializer': {'class_name': 'RandomUniform', 'config': {'minval': 0, 'maxval': None, 'seed': None, 'dtype': 'float32'}}, 'embeddings_regularizer': None, 'activi

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/60
10048/51069 [====>.........................] - ETA: 2:24 - loss: 2.0828 - acc: 0.3889